In [1]:
%reload_ext autoreload
%autoreload 2

In [3]:
!pip3 install py-algorand-sdk

In [4]:
import base64
import json
from algosdk.v2client import algod
from algosdk import account, mnemonic
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.transaction import *
from algod import *

In [5]:
# For ease of reference, add account public and private keys to
# an accounts dict.
accounts = {}
counter = 1
for m in [create_account(),create_account()]:
    accounts[counter] = {}
    accounts[counter]['public_key'] = m.get('public_key')
    accounts[counter]['private_key'] = m.get('private_key')
    counter += 1

In [6]:
import pprint
pprint.pprint(accounts)

{1: {'private_key': '3bsyqZN1SS5E+yIur13yQdW2WAawqTItWxVSaGAviXZ3jW8eS3lpJ0Vwztt/Kx9pMJFkcxq+U1iCv/pdYqyN+Q==',
     'public_key': 'O6GW6HSLPFUSORLQZ3NX6KY7NEYJCZDTDK7FGWECX75F2YVMRX4WRAUC5Y'},
 2: {'private_key': 'SnGHnmdlB4YLL4IzNC/6+cU0eP1q5rv+eK4t7QnHSGOvZsViDobWdWhL8tWs6lHasC9iQTsxq+uWc2uqsQGlhw==',
     'public_key': 'V5TMKYQOQ3LHK2CL6LK2Z2SR3KYC6YSBHMY2X24WONV2VMIBUWDQUD2NUE'}}


## Ran this command to transfer some microalgo to account 1 & 2
./sandbox goal clerk send -a 1000000 -f PPTVLKRE5DMIO42TYHGM6SMPMTB3A6V5PKV7JWFHRAQLOSCFPK3CEUXHJ4 -t public_key1  
./sandbox goal clerk send -a 1000000 -f PPTVLKRE5DMIO42TYHGM6SMPMTB3A6V5PKV7JWFHRAQLOSCFPK3CEUXHJ4 -t public_key2


In [7]:
account_info = algod_client.account_info(accounts[1]['public_key'])
print(f"Account balance for {accounts[1]['public_key']}: {account_info.get('amount')} microAlgos")
account_info = algod_client.account_info(accounts[2]['public_key'])
print(f"Account balance for {accounts[2]['public_key']}: {account_info.get('amount')} microAlgos")


Account balance for O6GW6HSLPFUSORLQZ3NX6KY7NEYJCZDTDK7FGWECX75F2YVMRX4WRAUC5Y: 1000000000000 microAlgos
Account balance for V5TMKYQOQ3LHK2CL6LK2Z2SR3KYC6YSBHMY2X24WONV2VMIBUWDQUD2NUE: 1000000000000 microAlgos


In [ ]:
# Get network params for transactions before every transaction.
params = algod_client.suggested_params()

# Account 1 creates an asset called graduation certificate 
# Ensure sender's private key is included
txid = create_asset(
    unit_name='CERT',
    asset_name='Grad_Cert_For_Trainee_Abhishek',
    total=1,
    sender_public_key=accounts[1]['public_key'],
    sender_private_key=accounts[1]['private_key'],  # Include the private key
    asset_url='https://tinyurl.com/eddt397j'
)




transaction {_struct:{} Sig:[23 204 171 246 9 239 88 170 139 142 51 44 118 63 207 142 252 175 255 251 150 214 255 27 183 39 62 131 17 56 171 9 93 84 238 242 208 17 193 89 57 216 200 253 86 201 181 121 159 207 247 77 11 190 233 137 183 21 219 248 74 170 90 2] Msig:{_struct:{} Version:0 Threshold:0 Subsigs:[]} Lsig:{_struct:{} Logic:[] Sig:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] Msig:{_struct:{} Version:0 Threshold:0 Subsigs:[]} Args:[]} Txn:{_struct:{} Type:acfg Header:{_struct:{} Sender:O6GW6HSLPFUSORLQZ3NX6KY7NEYJCZDTDK7FGWECX75F2YVMRX4WRAUC5Y Fee:{Raw:1000} FirstValid:152 LastValid:1152 Note:[] GenesisID:dockernet-v1 GenesisHash:J2EY4TYVNR5KF3PP6TSMH4LBQ6BYH65SR3LKS3P2HQ27RZFR7O5A Group:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA Lease:[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] RekeyTo:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAY5HFKQ} KeyregTxnFields:{_s

In [9]:
#   Utility function used to print created asset for account and assetid
def print_created_asset(algodclient, account, assetid):    
    # note: if you have an indexer instance available it is easier to just use this
    # response = myindexer.accounts(asset_id = assetid)
    # then use 'account_info['created-assets'][0] to get info on the created asset
    account_info = algodclient.account_info(account)
    idx = 0;
    for my_account_info in account_info['created-assets']:
        scrutinized_asset = account_info['created-assets'][idx]
        idx = idx + 1       
        if (scrutinized_asset['index'] == assetid):
            print("Asset ID: {}".format(scrutinized_asset['index']))
            print(json.dumps(my_account_info['params'], indent=4))
            break

#   Utility function used to print asset holding for account and assetid
def print_asset_holding(algodclient, account, assetid):
    # note: if you have an indexer instance available it is easier to just use this
    # response = myindexer.accounts(asset_id = assetid)
    # then loop thru the accounts returned and match the account you are looking for
    account_info = algodclient.account_info(account)
    idx = 0
    for my_account_info in account_info['assets']:
        scrutinized_asset = account_info['assets'][idx]
        idx = idx + 1        
        if (scrutinized_asset['asset-id'] == assetid):
            print("Asset ID: {}".format(scrutinized_asset['asset-id']))
            print(json.dumps(scrutinized_asset, indent=4))
            break


In [10]:
try:
    # Pull account info for the creator
    # account_info = algod_client.account_info(accounts[1]['pk'])
    # get asset_id from tx
    # Get the new asset's information from the creator account
    ptx = algod_client.pending_transaction_info(txid)
    asset_id = ptx["asset-index"]
    print_created_asset(algod_client, accounts[1]['public_key'], asset_id)
    print_asset_holding(algod_client, accounts[1]['public_key'], asset_id)
except Exception as e:
    print(e)

Asset ID: 1040
{
    "clawback": "FRE47I6OIB2ZNZPHM7JO56PPPU4C7RDFQCMVSAVIGVD5AVSXRPJEKOTGFM",
    "creator": "FRE47I6OIB2ZNZPHM7JO56PPPU4C7RDFQCMVSAVIGVD5AVSXRPJEKOTGFM",
    "decimals": 0,
    "default-frozen": false,
    "freeze": "FRE47I6OIB2ZNZPHM7JO56PPPU4C7RDFQCMVSAVIGVD5AVSXRPJEKOTGFM",
    "manager": "FRE47I6OIB2ZNZPHM7JO56PPPU4C7RDFQCMVSAVIGVD5AVSXRPJEKOTGFM",
    "name": "Grad_Cert_For_Trainee_Abdulhamid",
    "name-b64": "R3JhZF9DZXJ0X0Zvcl9UcmFpbmVlX0FiZHVsaGFtaWQ=",
    "reserve": "FRE47I6OIB2ZNZPHM7JO56PPPU4C7RDFQCMVSAVIGVD5AVSXRPJEKOTGFM",
    "total": 1,
    "unit-name": "ABDUCERT",
    "unit-name-b64": "QUJEVUNFUlQ=",
    "url": "https://ipfs.io/ipfs/Qmaq9wd5Pvfcie2rDxtafrj2UjssWL6AtCr2HYbXewwAzc",
    "url-b64": "aHR0cHM6Ly9pcGZzLmlvL2lwZnMvUW1hcTl3ZDVQdmZjaWUyckR4dGFmcmoyVWpzc1dMNkF0Q3IySFliWGV3d0F6Yw=="
}
Asset ID: 1040
{
    "amount": 1,
    "asset-id": 1040,
    "is-frozen": false
}


In [12]:
# OPT-IN

# Check if asset_id is in account 2's asset holdings prior
# to opt-in
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
# params.fee = 1000
# params.flat_fee = True

account_info = algod_client.account_info(accounts[2]['public_key'])
holding = None
idx = 0
for my_account_info in account_info['assets']:
    scrutinized_asset = account_info['assets'][idx]
    idx = idx + 1    
    if (scrutinized_asset['asset-id'] == asset_id):
        holding = True
        break

if not holding:
     # Use the AssetTransferTxn class to transfer assets and opt-in
    txn = AssetTransferTxn(
        sender=accounts[2]['public_key'],
        sp=params,
        receiver=accounts[2]["public_key"],
        amt=0,
        index=asset_id)
    stxn = txn.sign(accounts[2]['private_key'])
    # Send the transaction to the network and retrieve the txid.
    try:
        txid = algod_client.send_transaction(stxn)
        print("Signed transaction with txID: {}".format(txid))
        # Wait for the transaction to be confirmed
        confirmed_txn = wait_for_confirmation(algod_client, txid, 4) 
        print("TXID: ", txid)
        print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))
 
    except Exception as err:
        print(err)
    # Now check the asset holding for that account.
    # This should now show a holding with a balance of 0.
    print_asset_holding(algod_client, accounts[2]['public_key'], asset_id)
    



Signed transaction with txID: BL22WCKWETUZ4T63NKXHIPOLNGBF7DQF6ELZBBUNHPCRISKDDUJA
TXID:  BL22WCKWETUZ4T63NKXHIPOLNGBF7DQF6ELZBBUNHPCRISKDDUJA
Result confirmed in round: 6159
Asset ID: 1040
{
    "amount": 0,
    "asset-id": 1040,
    "is-frozen": false
}


In [13]:
# transfer asset of 10 from account 1 to account 3
params = algod_client.suggested_params()
# comment these two lines if you want to use suggested params
# params.fee = 1000
# params.flat_fee = True
txn = AssetTransferTxn(
    sender=accounts[1]['public_key'],
    sp=params,
    receiver=accounts[2]["public_key"],
    amt=1,
    index=asset_id)
stxn = txn.sign(accounts[1]['private_key'])
# Send the transaction to the network and retrieve the txid.
try:
    txid = algod_client.send_transaction(stxn)
    print("Signed transaction with txID: {}".format(txid))
    # Wait for the transaction to be confirmed
    confirmed_txn = wait_for_confirmation(algod_client, txid, 4) 
    print("TXID: ", txid)
    print("Result confirmed in round: {}".format(confirmed_txn['confirmed-round']))

except Exception as err:
    print(err)
# The balance should now be 10.
print_asset_holding(algod_client, accounts[2]['public_key'], asset_id)

Signed transaction with txID: 7ZAUA24L5UUC3WNDCKWW53PTYDVFQJTK3YXCSLNJ6YJRKKCABTMQ
TXID:  7ZAUA24L5UUC3WNDCKWW53PTYDVFQJTK3YXCSLNJ6YJRKKCABTMQ
Result confirmed in round: 6163
Asset ID: 1040
{
    "amount": 1,
    "asset-id": 1040,
    "is-frozen": false
}
